In [1]:
import sys
sys.path.insert(0, '..')

import time
import torch
import numpy as np
import pandas as pd
from fastprogress import master_bar, progress_bar

import src.utils as utils
import src.models as models
from src.config import config

import warnings
warnings.filterwarnings('ignore')

In [2]:
fold = 0
seed = 2020
num_epochs = 50
device = 'cuda' if torch.cuda.is_available() else 'cpu'
threshold = 0.5

utils.set_seed(seed)

In [3]:
df = utils.get_metadata(config)
splitter = utils.get_split(config)

df['fold'] = -1
for fold_id, (train_idx, valid_idx) in enumerate(splitter.split(df, df['ebird_code'])):
    df.iloc[valid_idx, -1] = fold_id
    
train_files = df.query('fold != @fold')[['file_path', 'ebird_code', 'duration']].values.tolist()
valid_files = df.query('fold == @fold')[['file_path', 'ebird_code', 'duration']].values.tolist()

In [4]:
train_loader, val_loader = utils.get_loaders(config, train_files, valid_files)

model = models.get_model(config)
model = model.to(device)

loss_func = utils.get_criterion(config)
optimizer = utils.get_optimizer(config, model)
scheduler = utils.get_scheduler(config, optimizer)

In [5]:
def mixup_data(x, y, alpha=1.0):
    if alpha > 0.:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1.
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).cuda()

    mixed_x = lam * x + (1 - lam) * x[index,:]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(y_a, y_b, lam):
    return lambda criterion, pred: lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

In [ ]:
best_F1 = -1
history = pd.DataFrame(columns=['epoch', 'train_loss', 'val_loss', 'F1_score', 'elapsed'])    

mb = master_bar(range(num_epochs))
for epoch in mb:
    start_time = time.time()
    model.train()
    avg_train_loss = 0
    for images, labels in progress_bar(train_loader, parent=mb):
        inputs, targets_a, targets_b, lam = mixup_data(images.to(device), labels.to(device), 0.2)
        outputs = model(inputs)
        criterion = mixup_criterion(targets_a, targets_b, lam)
        loss = criterion(loss_func, outputs)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        avg_train_loss += loss.item() / len(train_loader)

    model.eval()
    avg_valid_loss = 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in progress_bar(val_loader, parent=mb):
            preds = model(images.to(device))
            loss = loss_func(preds, labels.to(device))
                
            y_true.append(labels.cpu().detach())
            y_pred.append(preds.cpu().detach())
            avg_valid_loss += loss.item() / len(val_loader)
                
    scheduler.step()   
    y_true = torch.cat(y_true)
    y_pred = torch.cat(y_pred)
    F1 = utils.get_score(config, y_pred, y_true, threshold)
    if F1 > best_F1:
        best_F1 = F1
        torch.save({'epoch': epoch, 'model_state_dict': model.state_dict(), 'optimizer_state_dict': optimizer.state_dict()}, f'best_model.pth')
        
    F1 = round(F1, 4)
    avg_train_loss = round(avg_train_loss, 6)
    avg_valid_loss = round(avg_valid_loss, 6)
    elapsed = round(time.time() - start_time, 2)

    print(f'\nEpoch {epoch}:')
    print(f'Train Loss: {avg_train_loss} -- Valid Loss: {avg_valid_loss} -- F1 Score: {F1} -- Elapsed: {elapsed}')
    stats = {'epoch': epoch, 'train_loss': avg_train_loss, 'val_loss': avg_valid_loss, 'F1_score': F1, 'elapsed': elapsed}
    history = history.append(stats, ignore_index=True)

In [ ]:
history.to_csv('results.csv', index=False)